In [81]:

s3 = boto3.resource('s3')
bucket_name = 'nr-wsip'

index_prefix = "index_output/"
data_prefix = "data_output/"
min_alive_hours = 4

index_df = s3_to_df(bucket_name, index_prefix, parse_index_row)
data_df = s3_to_df(bucket_name, data_prefix, parse_data_row)



In [127]:
posts = index_df.join(data_df.set_index('id'), on='id')
# posts = posts.assign(alive_time=(posts['last_updated'] - posts['created_utc']))

# posts = posts[posts['alive_time'] > pd.Timedelta(hours=min_alive_hours)]

subs = posts['subreddit'].unique()

sub_posts = {sub: posts[posts["subreddit"] == sub] for sub in subs}

In [8]:
def posts_to_data(posts):
    return {
        "time_of_day": get_time_of_day_data(posts),
        "words": get_word_probs(posts)
    }

sub_data = {sub: posts_to_data(s_posts) for sub, s_posts in sub_posts.items()}
with open('sub_data.json', 'w') as outfile:
    json.dump(sub_data, outfile)

In [126]:
sub_posts['The_Donald']['created_utc'].sort_values().iloc[12400:12450]

319984   2017-05-25 14:45:30
320034   2017-05-25 14:45:31
320084   2017-05-25 14:45:35
320484   2017-05-25 14:45:54
322034   2017-05-25 14:47:15
322084   2017-05-25 14:47:16
322434   2017-05-25 14:47:25
322784   2017-05-25 14:47:43
323184   2017-05-25 14:48:01
323434   2017-05-25 14:48:10
323534   2017-05-25 14:48:13
323834   2017-05-25 14:48:21
324084   2017-05-25 14:48:27
324234   2017-05-25 14:48:31
318835   2017-05-25 14:49:08
318885   2017-05-25 14:49:09
318985   2017-05-25 14:49:13
319285   2017-05-25 14:49:38
319685   2017-05-25 14:49:51
88022    2017-05-26 00:00:03
88122    2017-05-26 00:00:11
88422    2017-05-26 00:00:19
88472    2017-05-26 00:00:34
88722    2017-05-26 00:00:44
88822    2017-05-26 00:00:54
89722    2017-05-26 00:02:05
89872    2017-05-26 00:02:15
90072    2017-05-26 00:02:30
90122    2017-05-26 00:02:30
90572    2017-05-26 00:02:41
84823    2017-05-26 00:03:13
84873    2017-05-26 00:03:15
84923    2017-05-26 00:03:16
85073    2017-05-26 00:03:24
85173    2017-

In [116]:
sub_posts['AskReddit']['created_utc'].sort_values().iloc[11750:11800]

69015    2017-05-24 14:35:18
69115    2017-05-24 14:35:19
69415    2017-05-24 14:35:32
68425    2017-05-24 15:19:40
72682    2017-05-24 15:54:24
68889    2017-05-24 16:22:44
72449    2017-05-24 17:06:30
305255   2017-05-25 00:00:02
305555   2017-05-25 00:00:23
305455   2017-05-25 00:00:23
305855   2017-05-25 00:00:35
306005   2017-05-25 00:00:36
306405   2017-05-25 00:00:47
308405   2017-05-25 00:03:06
308505   2017-05-25 00:03:07
308455   2017-05-25 00:03:07
308755   2017-05-25 00:03:23
309155   2017-05-25 00:03:34
309205   2017-05-25 00:03:35
309305   2017-05-25 00:03:40
309705   2017-05-25 00:04:13
309805   2017-05-25 00:04:16
310205   2017-05-25 00:04:32
310405   2017-05-25 00:04:34
310355   2017-05-25 00:04:34
310755   2017-05-25 00:04:44
310905   2017-05-25 00:04:46
310805   2017-05-25 00:04:46
311255   2017-05-25 00:05:11
305106   2017-05-25 00:05:23
305356   2017-05-25 00:05:28
305556   2017-05-25 00:05:32
306056   2017-05-25 00:05:53
306756   2017-05-25 00:06:39
307056   2017-

In [128]:
get_time_of_day_data(sub_posts['AskReddit'])

{'bin': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23],
 'count': [3564,
  3661,
  3658,
  3495,
  3312,
  2970,
  2814,
  2448,
  2235,
  2242,
  2282,
  2648,
  3159,
  3384,
  3550,
  3661,
  3754,
  3649,
  3515,
  3567,
  3505,
  3505,
  3505,
  3506],
 'med': [2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  2.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0]}

In [80]:
import pandas as pd
import boto3
import json

def to_dates(keys):
    dates = []
    for key in keys:
        parts = key.split('/')
        if (parts[1] == ""):
            continue
        dates.append(parts[1])
    return set(dates)

def to_latest_output_keys(latest_output_time, output_keys):
    keys = []
    for key in output_keys:
        parts = key.split('/')
        if len(parts) < 3:
            continue
        if (parts[1] == latest_output_time) & (parts[2] != '_SUCCESS') & (parts[2] != 'manifest'):
            keys.append(key)
    return keys

def get_latest_output_keys(bucket_name, prefix):
    output_keys = [obj.key for obj in s3.Bucket(bucket_name).objects.filter(Prefix=prefix)]
    output_times = to_dates(output_keys) 
    latest_time = max(output_times)
    latest_output_keys = to_latest_output_keys(latest_time, output_keys)
    return latest_output_keys

def s3_to_df(bucket_name, prefix, parser):
    keys = get_latest_output_keys(bucket_name, prefix)
    output = []
    for key in keys:
        s3_obj = s3.Object(bucket_name, key).get()['Body'].read().decode('utf-8')
        rows = s3_obj.split("\n")[:-1]
        this_output = [parser(row) for row in rows]
        output += this_output
    return pd.DataFrame(output)

def to_ts(unix_str):
    return pd.Timestamp(int(unix_str), unit='s')

def get_string_prop(prop, row):
    return row[prop]['s']

def parse_index_row(row):
    output = {}
    row_dict = json.loads(row)
    output['created_utc'] = to_ts(row_dict['created_utc']['n'])
    output['last_updated'] = to_ts(row_dict['last_updated']['n'])
    output['day'] = row_dict['day']['s']
    output['id'] = row_dict['id']['s']
    output['score'] = int(row_dict['score']['n'])
    return output

def parse_data_row(row):
    row_dict = json.loads(row)
    props = ['id', 'author', 'title', 'permalink', 'selftext', 'subreddit']
    return {prop: get_string_prop(prop, row_dict) for prop in props}

In [95]:
def get_time_of_day_data(posts):
    
    
    created_times = posts['created_utc']
    created_hours = posts['created_utc'].apply(lambda x: x.hour)
    posts = posts.assign(hour=created_hours)
    bin_results = {'bin': [], 'med': [], 'count':[]}  
    for h in range(24):
        bin_posts = posts[posts["hour"] == h]
        if len(bin_posts):
                bin_med_score = bin_posts['score'].median()
        else:
            bin_med_score = 0
        bin_results['bin'].append(h)
        bin_results['med'].append(bin_med_score)
        bin_results['count'].append(len(bin_posts))
    
    return bin_results

In [88]:
def get_time_of_day_data_old(posts):

    def to_minute_of_day(timestamp):
        return (timestamp.hour * 60) + timestamp.minute
    created_times = posts['created_utc']
    created_minutes = [to_minute_of_day(t) for _, t in created_times.items()]

    bins = [30 * n for n in range(49)]
    group_names = range(48)
    time_bins = pd.cut(created_minutes, bins, labels=group_names)
    posts = posts.assign(time_bin=time_bins)

#     bin_results = {'bin': [], 'med': [], 'upper': [], 'lower': [], 'count':[]}
    bin_results = {'bin': [], 'med': [], 'count':[]}    
    for t in group_names:
        bin_posts = posts[posts["time_bin"] == t]
        bin_med_score = bin_posts['score'].median()
        if len(bin_posts):
            bin_med_score = bin_posts['score'].median()
        else:
            bin_med_score = 0
        bin_upper_score = bin_med_score + bin_posts['score'].std()
        bin_lower_score = bin_med_score + bin_posts['score'].std()
        bin_count = bin_posts['score'].count().item()
        bin_results['bin'].append(t)
        bin_results['med'].append(bin_med_score)
#         bin_results['upper'].append(bin_upper_score)
#         bin_results['lower'].append(bin_lower_score)
        bin_results['count'].append(bin_count)
    
    return bin_results

In [89]:
import math
from collections import defaultdict
import re

def to_message(row):
    is_good = row['good']
    words = "{0} {1}".format(row['title'], row['selftext'])
    return (words, is_good)

def classify_posts(lower, upper, score):
    if score >= upper:
        return True
    elif score < lower:
        return False
    else:
        return
    
def probs_to_distance(p_good, p_bad):
    return (p_good - p_bad)/math.sqrt(2)

def get_word_probs(posts):
    result = {
        "best": {"words": [], "probs": []}, "worst": {"words": [], "probs": []}
    }
#     lower_score = max([posts['score'].quantile(0.33), 2])
    lower_score = 3
    upper_score = max([posts['score'].quantile(0.66), 4])
    posts = posts.assign(
        good=posts['score'].apply(
            lambda p: classify_posts(lower_score, upper_score, p)
        )).dropna(subset=['good'])
    messages = [to_message(post) for _, post in posts.iterrows()]
    word_probs = messages_to_word_probabilities(messages)
    word_dists = [(word, probs_to_distance(p_good, p_bad)) for
                  word, p_good, p_bad in word_probs]

    if len(word_dists) == 0:
        return result
    
    words, dists = zip(*word_dists)
    word_dists = pd.Series(dists, index=words).sort_values()
    worst = word_dists.head(10)
    best = word_dists.tail(10).sort_values(ascending=False)
    result['best']['words'] = best.index.tolist()
    result['best']['probs'] = best.values.tolist()
    result['worst']['words'] = worst.index.tolist()
    result['worst']['probs'] = worst.values.tolist()
    return result

    
def tokenize(message):
    stops = ['our', 'their', 'your', 'one', 'about', "i'm", 'up', 'out', 'am', 'any', 'like', 'when', 'now', 'her', "it's",  'we', 'us', 'they', 'he', 'she', 'his', 'u', 'com', 'http', 'https', 'www', 'or', 'who', 'would', 'had', 'any' 'at', 'got', 'l', 'my', 'me', 'does','get', 'were', 'what', 'at', 'too', 'as', 's', 'an', 'than', 'do', 'so', 'no', 'it', 'how', 'be', 'has',  'a','can', 'will', 'have', 'if', 'why', 'but','he', 'the', 'and', 'on', 'all', 'is', 'emptystring', 'are', 'said', 'in', 'of', 'just', 'that', 'i', 'with', 'was', 'r', 't', 'to', 'for', 'by', 'you', 'there', 'not', 'to', 'from', 'this', '0' , '1', '2', '3', '4', '5', '6', '7', '8', '9']
    message = message.lower()                       # convert to lowercase
    all_words = re.findall("[a-z0-9']+", message)   # extract the words
    filt_words = list(filter(lambda x: x not in stops, all_words))
    
    return set(filt_words)   
    
def count_words(messages):
    """training set consists of pairs (message, is_spam)"""
    counts = defaultdict(lambda: [0, 0])
    for message, is_good in messages:
        for word in tokenize(message):
            counts[word][0 if is_good else 1] += 1
    return counts
    
def messages_to_word_probabilities(messages):
    num_goodposts = len([is_good
                     for message, is_good in messages
                     if is_good])
    num_non_goodposts = len(messages) - num_goodposts

    k = 0.5
    word_counts = count_words(messages)
    word_probs = word_probabilities(word_counts,
                                         num_goodposts,
                                         num_non_goodposts,
                                         k)
    return word_probs

def word_probabilities(counts, total_goodposts, total_non_goodposts, k=0.5):
    """turn the word_counts into a list of triplets
    w, p(w | good) and p(w | ~good)"""
    return [(w,
             (good + k) / (total_goodposts + 2 * k),
             (non_good + k) / (total_non_goodposts + 2 * k))
             for w, (good, non_good) in counts.items()]
